# A2A (Agent-to-Agent) Protocol Demo

This notebook demonstrates how to connect to an A2A server using the Microsoft Agent Framework.

**Prerequisites:**
- An A2A server running at `http://localhost:8000`
- The server should expose an agent card at `/.well-known/agent-card.json`

**Steps:**
1. Discover the agent via its Agent Card
2. Create an A2AAgent client
3. Send messages and receive responses

In [8]:
import asyncio
import httpx
from a2a.client import A2ACardResolver
from agent_framework.a2a import A2AAgent


async def main():
    a2a_host = "http://localhost:8000"

    # 1. Discover the remote agent's capabilities
    async with httpx.AsyncClient(timeout=60.0) as http_client:
        resolver = A2ACardResolver(httpx_client=http_client, base_url=a2a_host)
        agent_card = await resolver.get_agent_card()
        print(f"Found agent: {agent_card.name}")

    # 2. Create an A2AAgent with a session and send a message
    async with A2AAgent(
        name=agent_card.name,
        id=f"{agent_card.name}_client",
        agent_card=agent_card,
        url=a2a_host,
    ) as agent:
        # Create a session for the conversation
        session = agent.create_session()
        
        # Send a message using the session
        response = await agent.run("Hello! Tell me about yourself.", session=session)
        
        print("\n===== Response =====")
        for message in response.messages:
            print(message.text)
        
        # Send another message in the same session
        response2 = await agent.run("What can you help me with?", session=session)
        
        print("\n===== Second Response =====")
        for message in response2.messages:
            print(message.text)

In [9]:
await main()

Found agent: Hello World LangGraph Agent


A2AClientJSONRPCError: JSON-RPC Error code=-32602 data=None message='1 validation error for TaskSendParams\nid\n  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.9/v/string_type'

## Multi-Turn Conversation Example

The following cell demonstrates multiple turns in a conversation with the A2A agent:

In [12]:
async def multi_turn_example():
    a2a_host = "http://localhost:8000"
    
    async with httpx.AsyncClient(timeout=60.0) as http_client:
        resolver = A2ACardResolver(httpx_client=http_client, base_url=a2a_host)
        agent_card = await resolver.get_agent_card()
        print(f"Agent: {agent_card.name}\n")
    
    async with A2AAgent(
        name=agent_card.name,
        id=f"{agent_card.name}_multi_turn_client",
        agent_card=agent_card,
        url=a2a_host,
    ) as agent:
        session = agent.create_session()
        
        # First message
        print("User: What's your purpose?")
        response1 = await agent.run("What's your purpose?", session=session)
        print(f"Agent: {response1.text}\n")
        
        # Second message (continues the conversation)
        print("User: Can you give me an example?")
        response2 = await agent.run("Can you give me an example?", session=session)
        print(f"Agent: {response2.text}\n")
        
        # Third message
        print("User: Thank you!")
        response3 = await agent.run("Thank you!", session=session)
        print(f"Agent: {response3.text}\n")

await multi_turn_example()

Agent: Hello World LangGraph Agent

User: What's your purpose?


A2AClientJSONRPCError: JSON-RPC Error code=-32602 data=None message='1 validation error for TaskSendParams\nid\n  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]\n    For further information visit https://errors.pydantic.dev/2.9/v/string_type'